In [1]:
#!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook

try:
    from collections import OrderedDict
except ImportError:
    from ordereddict import OrderedDict

# Torch, Sklearn imports
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler

## NLP libs
from nltk import download
import gensim

## PyTorch Transformer
import transformers

## Roberta
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

## DistilBert
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import DistilBertForSequenceClassification, DistilBertConfig
from transformers.optimization import AdamW, WarmupLinearSchedule

import warnings
warnings.filterwarnings("ignore")
stopwords = {"ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than"}

print(torch.__version__)
print(transformers.__version__)

1.1.0
2.1.1


In [3]:
!ls

1.text_classifier_roberta.ipynb      4.nips2019_papers_simple_altair.ipynb
1.text_classifier_roberta_NEW.ipynb  4.sample_dpp.ipynb
2017-06-custom-intent-engines	     intents_phrases_183.pkl
2.uncertainty_swag.ipynb	     intents_phrases_186.pkl
3.causality_review.ipynb	     model_elmo_swag_uncertainty.pth
4.dpp_diversity_phrases.ipynb	     nips_2018_bert.pkl
4.dpp_image.ipynb		     nips_2018_elmo.pkl
4.nips2019_papers.ipynb		     nips_2018.pkl


In [4]:
# dataset_path = "intents_phrases_186.pkl"
# dataset = pd.read_pickle(dataset_path)
# dataset = dataset.rename(columns={"usersays":"phrase"})
# dataset.tail()

In [5]:
# dataset.intent.value_counts()

In [6]:
# # Make shorter version of the dataset
# selected_intents = ['position.update',
#                     'jobBoard.update',
#                     'job.create',
#                     'lateralMove',
#                     'band.update',
#                     'adjustment',
#                    'worker.changeManager']
# dataset = dataset[dataset.intent.isin(selected_intents)].reset_index(drop=True)
# print(len(set(dataset.intent)))
# dataset.tail()

In [7]:
dataset_path = './2017-06-custom-intent-engines/'
## Another dataset
dataset = pd.DataFrame(columns = ['utterance', 'label'])
for intent in ['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork',
               'SearchScreeningEvent']:
    with open(dataset_path + intent + "/train_" + intent + ".json",
              encoding='cp1251') as data_file:
        data = json.load(data_file)
    print("Class: {}, # utterances: {}".format(intent,len(data[intent])))
    texts = []
    for i in range(len(data[intent])):
        text = ''
        for j in range(len(data[intent][i]['data'])):
            text += data[intent][i]['data'][j]['text']
        dataset = dataset.append({'utterance': text, 'label': intent}, ignore_index=True)
dataset = dataset.rename(columns={"utterance":"phrase", "label":"intent"})
dataset.tail()

Class: AddToPlaylist, # utterances: 300
Class: BookRestaurant, # utterances: 300
Class: GetWeather, # utterances: 300
Class: PlayMusic, # utterances: 300
Class: RateBook, # utterances: 300
Class: SearchCreativeWork, # utterances: 300
Class: SearchScreeningEvent, # utterances: 300


,phrase,intent
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent


## Over-sampling dataset

In [8]:
def resample_dataset(dataframe,
                     label_column = 'intent',
                     feature_column = 'phrase',
                     max_samples = 100):
    
    from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN

    ## Build label vocabulary
    label_to_ix = {}
    for label in dataset[label_column]:
        for word in label.split():
            if word not in label_to_ix:
                label_to_ix[word]=len(label_to_ix)
          
    ## Define Sampling Strategy based on number of samples
    classes_sample = {}
    for cls in list(set(label_to_ix.values())):
        classes_sample.update({cls:max_samples})

    sampler = RandomOverSampler(sampling_strategy = classes_sample, random_state=42)
    x = np.array(dataset.index).reshape(-1, 1)
    y = np.array(list(dataset[label_column].apply(lambda x: label_to_ix[x])))
    
    ## Oversampling
    x_resampled, y_resampled = sampler.fit_sample(x, y)
    dataset_resampled = pd.DataFrame(columns=[feature_column,label_column])
    
    ## Iterating
    for i, item in enumerate(x_resampled):
        row = {
            feature_column :dataset[feature_column].loc[item[0]],
            label_column: list(label_to_ix.keys())[y_resampled[i]]
        }
        dataset_resampled = dataset_resampled.append(row, ignore_index=True)
    return dataset_resampled

In [9]:
# # dataset = resample_dataset(dataset, max_samples = 300)
# dataset.intent.value_counts()

In [10]:
dataset.tail()

,phrase,intent
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent


## Cleaning Dataset

In [11]:
def transformText(text, do_stop=False, do_stem=False):
    # Convert text to lower
    text = text.lower()
    
    # Cleaning input
    text = text.replace("'s","")
    text = text.replace("’s","")
    text = text.replace("?","")
    text = text.replace("-","")
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    # Removing all the stopwords
    if (do_stop==True):
        filtered_words = [word for word in text.split() if word not in stopwords]
    else:
        filtered_words = [word for word in text.split()]
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    if (do_stem==True):
        # Stemming
        text = gensim.parsing.preprocessing.stem_text(text)
    return text

In [12]:
dataset['preproc_text'] = dataset['phrase'].apply(lambda x: transformText(x, do_stop=True))
dataset.tail(5)

,phrase,intent,preproc_text
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent,across line playing closest movie house
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent,animated movies playing neighbourhood
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent,always return dawn playing
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent,movie schedule neighborhood
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent,tell howling ii sister werewolf playing


In [13]:
## Build label vocabulary
label_to_ix = {}
for label in dataset.intent:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
print("# Labels: {}".format(len(label_to_ix)))

# Labels: 7


In [14]:
model_type = 'roberta'
## Distilbert
if model_type == 'distilbert':
    print("DistilBERT")
    config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
    config.num_labels = len(list(label_to_ix.values()))
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification(config)
    print(config)
elif model_type == 'roberta':
    print("RoBERTa")
    config = RobertaConfig.from_pretrained('roberta-base')
    config.num_labels = len(list(label_to_ix.values()))
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaForSequenceClassification(config)
    print(config)

RoBERTa
{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 7,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "vocab_size": 50265
}



## New Model 

In [15]:
def prepare_features(seq_1, zero_pad = False, max_seq_length = 300):
    enc_text = tokenizer.encode_plus(seq_1, add_special_tokens=True, max_length=300)
    if zero_pad:
        while len(enc_text['input_ids']) < max_seq_length:
            enc_text['input_ids'].append(0)
            enc_text['token_type_ids'].append(0)
    return enc_text

In [16]:
prepare_features("testing this loved", zero_pad = True)

{'special_tokens_mask': [1, 0, 0, 0, 1],
 'input_ids': [0,
  33959,
  42,
  2638,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [17]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        utterance = self.data.preproc_text[index]
        label = self.data.intent[index]
        X = prepare_features(utterance, zero_pad = True)
        y = label_to_ix[self.data.intent[index]]
        return np.array(X['input_ids']), np.array(X['token_type_ids']), y
    
    def __len__(self):
        return self.len

In [18]:
dataset.tail(10)

,phrase,intent,preproc_text
2090,I want to see a list of the closest cinema's m...,SearchScreeningEvent,want see list closest cinema movies
2091,What Are the showings for The Natural History ...,SearchScreeningEvent,showings natural history parking lots movie house
2092,Give me the schedule for Public Stenographer a...,SearchScreeningEvent,give schedule public stenographer mjr theatres
2093,Is it possible to see Tube at the closest movi...,SearchScreeningEvent,possible see tube closest movie theatre
2094,I want to see Wenn Lucy springt now at a movie...,SearchScreeningEvent,want see wenn lucy springt movie theatre
2095,Is Across the Line playing at the closest movi...,SearchScreeningEvent,across line playing closest movie house
2096,Which animated movies are playing in the neigh...,SearchScreeningEvent,animated movies playing neighbourhood
2097,Where is They Always Return at Dawn playing,SearchScreeningEvent,always return dawn playing
2098,What is the movie schedule in the neighborhood,SearchScreeningEvent,movie schedule neighborhood
2099,Tell me when Howling II: Your Sister Is a Were...,SearchScreeningEvent,tell howling ii sister werewolf playing


In [19]:
train_size = 0.8
dataset = pd.concat([dataset, dataset]).reset_index(drop=True)
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.tail()

,phrase,intent,preproc_text
4195,Add Fair Charlotte to the We Everywhere playlist.,AddToPlaylist,add fair charlotte everywhere playlist
4196,Start playing anything Mike Muir made in the t...,PlayMusic,start playing anything mike muir made thirties
4197,"Please look up for the work, Black Box.",SearchCreativeWork,please look work black box
4198,give this textbook a four,RateBook,give textbook four
4199,Book me a reservation for six for Crown Burger...,BookRestaurant,book reservation six crown burgers lebanon


In [20]:
train_dataset=dataset.sample(frac=train_size,random_state=42).reset_index(drop=True)
test_dataset=dataset.drop(train_dataset.index).reset_index(drop=True)

In [21]:
training_set = Intents(train_dataset)
testing_set = Intents(test_dataset)

In [22]:
train_dataset.intent.value_counts()

PlayMusic               496
SearchScreeningEvent    490
GetWeather              482
RateBook                481
SearchCreativeWork      479
AddToPlaylist           467
BookRestaurant          465
Name: intent, dtype: int64

In [23]:
test_dataset.intent.value_counts()

SearchScreeningEvent    135
PlayMusic               127
SearchCreativeWork      120
GetWeather              119
RateBook                116
BookRestaurant          115
AddToPlaylist           108
Name: intent, dtype: int64

In [24]:
### Dataloaders Parameters
params = {'batch_size': 2,
          'shuffle': True,
          'drop_last': True,
          'num_workers': 0}
training_loader = DataLoader(training_set, **params)
testing_loader = DataLoader(testing_set, **params)
loss_function = nn.CrossEntropyLoss()
learning_rate = 2e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)
if torch.cuda.is_available():
    print("GPU is AVAILABLE!🤘🙌💪")
    model = model.cuda()

GPU is AVAILABLE!🤘🙌💪


In [25]:
ids, tokens, labels = next(iter(training_loader))
ids.shape, tokens.shape, labels.shape

(torch.Size([2, 300]), torch.Size([2, 300]), torch.Size([2]))

In [26]:
ids.shape

torch.Size([2, 300])

In [27]:
if model_type == 'roberta':
    print("RoBERTa")
    out = model.forward(ids.cuda(), token_type_ids=tokens.cuda(), head_mask=None)[0]
elif model_type == 'distilbert':
    print("DistilBERT")
    out = model.forward(ids.cuda())[0]
print(loss_function(out, labels.cuda()))
print(out.shape)

RoBERTa
tensor(1.7798, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 7])


In [28]:
labels

tensor([4, 2])

In [ ]:
max_epochs = 5
model = model.train()
for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (ids, tokens, labels) in enumerate(training_loader):
        optimizer.zero_grad()
        if torch.cuda.is_available():
            ids = ids.cuda()
            tokens = tokens.cuda()
            labels = labels.cuda()
        if model_type == 'roberta':
            output = model.forward(ids,token_type_ids=tokens)[0]
        elif model_type == 'distilbert':
            output = model.forward(ids)[0]
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        if i%100 == 0:
            correct = 0
            total = 0
            for (ids, tokens, labels) in testing_loader:
                if torch.cuda.is_available():
                    ids = ids.cuda()
                    tokens = tokens.cuda()
                    labels = labels.cuda()
                if model_type == 'roberta':
                    output = model.forward(ids,token_type_ids=tokens)[0]
                elif model_type == 'distilbert':
                    output = model.forward(ids)[0]
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
            accuracy = 100.00 * correct.numpy() / total
            print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

EPOCH -- 0
Iteration: 0. Loss: 1.8866183757781982. Accuracy: 15.238095238095237%
Iteration: 100. Loss: 2.311615228652954. Accuracy: 14.166666666666666%
Iteration: 200. Loss: 1.6845951080322266. Accuracy: 12.738095238095237%
Iteration: 300. Loss: 2.2933998107910156. Accuracy: 14.166666666666666%
Iteration: 400. Loss: 1.884190559387207. Accuracy: 13.333333333333334%
Iteration: 500. Loss: 2.0379934310913086. Accuracy: 13.69047619047619%
Iteration: 600. Loss: 2.385106086730957. Accuracy: 14.166666666666666%
Iteration: 700. Loss: 1.9154937267303467. Accuracy: 14.404761904761905%
Iteration: 800. Loss: 2.3012232780456543. Accuracy: 13.69047619047619%
Iteration: 900. Loss: 1.8624022006988525. Accuracy: 17.261904761904763%
Iteration: 1000. Loss: 1.983472466468811. Accuracy: 13.333333333333334%
Iteration: 1100. Loss: 2.1209373474121094. Accuracy: 14.285714285714286%
Iteration: 1200. Loss: 2.0511763095855713. Accuracy: 15.833333333333334%
Iteration: 1300. Loss: 1.8092669248580933. Accuracy: 14.76

## Parameters

| Model     | LR       | Batch Size   | Epochs    | Accuracy    |
| :---:     | :---:    | :---:        | :---:     |  :---:      |
| RoBERTa   |   1e-05  |     2        |    5      |   97.73%    | 
| RoBERTa   |   1e-05  |     4        |    5      |   98.57%    | 
| RoBERTa   |   2e-05  |     2        |    5      |          | 
| RoBERTa   |   2e-05  |     4        |    5      |          | 



In [ ]:
msg = "radiohead playlist"
input_msg = prepare_features(msg, zero_pad=True)
ids = torch.tensor(input_msg['input_ids'])
tokens = torch.tensor(input_msg['token_type_ids'])
ids
out = model.forward(ids.cuda(),token_type_ids=tokens.cuda())

#model.forward(input_msg['input_ids'],token_type_ids=input_msg['token_type_ids'])

In [ ]:
def get_reply(msg):
    model.eval()
    input_msg, _ = prepare_features(msg)
    if torch.cuda.is_available():
        input_msg = input_msg.cuda()
    output = model(input_msg)[0]
    _, pred_label = torch.max(output.data, 1)
    prediction=list(label_to_ix.keys())[pred_label]
    return prediction